# 一、导入数据

In [230]:
import numpy as np
import pandas as pd
import scipy.stats as stats

GDP = pd.read_excel('CME_Qqgdp.xlsx').loc[2:,:]
OPI = pd.read_excel('FS_Comscfd.xlsx').loc[2:,:]
ret = pd.read_excel('TRD_Mnth.xlsx').loc[2:,:]

GDP = GDP[GDP['Quarter']>='2005-03']
OPI = OPI[(OPI['Stkcd']<'700000')&(OPI['Stkcd']>='600000')]
GDP['Quarter'] = GDP['Quarter'].apply(lambda x: int(x[0:4]+x[5:7]))
OPI['Quarter'] = OPI['Accper'].apply(lambda x: int(x[0:4]+x[5:7]) if x[5:7]!='01' else int(x[0:4]+x[5:7])-89)
OPI = OPI[~OPI['ShortName'].isin(OPI[OPI['C001000000']==0]['ShortName'].tolist())]
OPI = OPI.drop_duplicates(subset=['Stkcd','Quarter'])

ret['Newmnt'] = ret['Trdmnt'].apply(lambda x: int(x.replace('-','')))
ret['NewMretwd'] = ((1+ret['Mretwd'])*(1+ret['Mretwd'].shift(1))*(1+ret['Mretwd'].shift(2))-1).astype('float64') #计算季度累计收益率

C:\ProgramData\Anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\ProgramData\Anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\ProgramData\Anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# 二、计算12个季度的LL因子值

In [231]:
#生成样本所需时间序列，20个季度用来计算相关系数,8个季度的加权，12个季度的验证
start_date = 201003  #样本形成期的季度
formation_dates=[start_date]
for i in range(1,40):
    if formation_dates[-1]%10==2:
        formation_dates.append(formation_dates[-1]+91)
    else:
        formation_dates.append(formation_dates[-1]+3)

#筛选时间序列里全部有值的股票
OPI['flag'] = OPI['Quarter'].apply(lambda x: 1 if x in formation_dates else 0)
temp1 = OPI.groupby('Stkcd')['flag'].sum()
stocks_list = temp1[temp1==len(formation_dates)].index.tolist()

OPI = OPI[OPI['Stkcd'].isin(stocks_list)]
OPI.loc[:,'OPI_gr'] = OPI['C001000000']/OPI['C001000000'].shift(4)#直接计算增长率

#最开始的4期为nan
OPI['flag1'] = OPI['Quarter']<OPI['Quarter'].shift(4)
OPI.loc[OPI[OPI['flag1']==True].index,'OPI_gr'] = np.nan

#保证样本期内的值不为nan
stocks_list = list(set(stocks_list).intersection(set(OPI[(OPI['Quarter']==start_date)&(OPI['OPI_gr']).notnull()]['Stkcd'].tolist())))

allLL_factor = pd.DataFrame()
for j in range(0,12):
    print('T0='+str(formation_dates[23+j]))
    #计算个股因子
    LL_factor = {}
    for stock in stocks_list:
        temp_opi = OPI[OPI['Stkcd']==stock]
        corr = [abs(stats.pearsonr(temp_opi[temp_opi['Quarter'].isin(formation_dates[4+i+j:24+i+j])]['OPI_gr'], GDP[GDP['Quarter'].isin(formation_dates[4+j:24+j])]['Gdpcurrperiod'])[0]) for i in range(-4,5)]
        LL_factor[stock] = -np.array(list(range(-4,5))).dot(np.array(corr))/np.array(corr).sum()

    LL_factor = pd.DataFrame(LL_factor,index=['LL_factor']).T
    LL_factor['Stkcd'] = LL_factor.index
    LL_factor['Newmnt'] = formation_dates[28+j]
    allLL_factor=pd.concat((allLL_factor,LL_factor),axis=0)

## 当前期的因子和下一期的return做合并
testdata=pd.merge(ret,allLL_factor.reset_index(),how='inner',on=['Stkcd','Newmnt'])
testdata

T0=201512
T0=201603
T0=201606
T0=201609
T0=201612
T0=201703
T0=201706
T0=201709
T0=201712
T0=201803
T0=201806
T0=201809


,Stkcd,Trdmnt,Mretwd,Newmnt,NewMretwd,index,LL_factor
0,600000,2017-03,-0.034961,201703,-0.012339,600000,-0.759847
1,600000,2017-06,-0.014798,201706,0.039391,600000,-0.379692
2,600000,2017-09,0.012589,201709,0.017391,600000,-0.385548
3,600000,2017-12,-0.024787,201712,-0.021756,600000,-0.080037
4,600000,2018-03,-0.065008,201803,-0.074662,600000,-0.270849
...,...,...,...,...,...,...,...
9701,601999,2018-12,-0.078182,201812,-0.034286,601999,0.674254
9702,601999,2019-03,0.05939,201903,0.301776,601999,0.922837
9703,601999,2019-06,-0.009646,201906,-0.066666,601999,0.713598
9704,601999,2019-09,-0.019197,201909,-0.070360,601999,0.543098


# 三、分位数组合LL因子有效性检验

In [232]:
import statsmodels.api as sm

def getgroup(x):
    return pd.qcut(x,10,labels=False)

#decile portfolios
testdata['group'] = testdata.groupby('Newmnt')['LL_factor'].apply(getgroup)
gret = testdata.groupby(['Newmnt','group'])['NewMretwd'].mean()
gretnew = gret.reset_index().pivot_table(index='Newmnt',columns='group',values='NewMretwd')
gretnew.loc[:,'spread']=gretnew.loc[:,9]-gretnew.loc[:,0]

from scipy import stats
print('单因素t检验：')
print(stats.ttest_1samp(gretnew['spread'], 0))

tmpdata=sm.add_constant(gretnew['spread'])
tmpdata

mod = sm.OLS(tmpdata.iloc[:,1], tmpdata.iloc[:,0])
reshac=mod.fit(cov_type='hac',cov_kwds={'maxlags':2}) #Newey-west estimator
print(reshac.summary())

单因素t检验：
Ttest_1sampResult(statistic=-0.13362664153390816, pvalue=0.896111183223277)
                            OLS Regression Results                            
Dep. Variable:                 spread   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 31 Oct 2023   Prob (F-statistic):                nan
Time:                        21:34:45   Log-Likelihood:                 28.874
No. Observations:                  12   AIC:                            -55.75
Df Residuals:                      11   BIC:                            -55.26
Df Model:                           0                                         
Covariance Type:                  hac                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------

C:\ProgramData\Anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# 四、收益对LL因子回归

In [233]:
def dofmreg(data):
    mod = sm.OLS(data.loc[:,'NewMretwd'], sm.add_constant(data.loc[:,'LL_factor']))
    res = mod.fit()
    return res.params

regres=testdata.groupby('Newmnt').apply(dofmreg)
print('单因素t检验：')
print(stats.ttest_1samp(regres['LL_factor'], 0))

单因素t检验：
Ttest_1sampResult(statistic=0.09655797308524354, pvalue=0.924814201038695)
